In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
from PIL import Image, ImageTk
import requests
from io import BytesIO
import random
import json
import os
from datetime import datetime
import threading
from queue import Queue
import time

class EnhancedCocktailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Коктейльный помощник PRO+")
        
        # Устанавливаем окно на весь экран с учетом масштабирования
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Настройка цветовой схемы
        self.bg_color = "#252525"
        self.secondary_bg = "#1E1E1E"
        self.text_color = "#FFFFFF"
        self.accent_color = "#4A89DC"
        self.highlight_color = "#5D9BFA"
        self.button_hover = "#3B7DD8"
        self.favorite_color = "#FFD700"
        
        # Шрифты
        self.title_font = ('Segoe UI', 18, 'bold')
        self.subtitle_font = ('Segoe UI', 14, 'bold')
        self.normal_font = ('Segoe UI', 13)
        self.button_font = ('Segoe UI', 13, 'bold')
        self.small_font = ('Segoe UI', 11)
        
        # Настройка стилей
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self._configure_styles()
        
        # Основной контейнер
        self.main_container = ttk.Frame(root, style='Main.TFrame')
        self.main_container.pack(fill=tk.BOTH, expand=True, padx=25, pady=25)
        
        # Холст с прокруткой
        self.canvas = tk.Canvas(
            self.main_container, 
            bg=self.bg_color,
            highlightthickness=0
        )
        self.scrollbar = ttk.Scrollbar(
            self.main_container, 
            orient="vertical", 
            command=self.canvas.yview,
            style='Vertical.TScrollbar'
        )
        
        # Основной фрейм
        self.scrollable_frame = ttk.Frame(
            self.canvas, 
            style='Secondary.TFrame',
            padding=(40, 30)
        )
        
        # Настройка прокрутки
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(
                scrollregion=self.canvas.bbox("all")
            )
        )
        
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Очередь для обработки изображений в отдельном потоке
        self.image_queue = Queue()
        self.image_cache = {}
        self.current_image_url = None
        
        # Запускаем поток для обработки изображений
        self.image_thread = threading.Thread(target=self._process_image_queue, daemon=True)
        self.image_thread.start()
        
        # Загрузка данных
        self.load_data()
        
        # Создание интерфейса
        self.create_widgets()
        
        # Обработчик изменения размера окна
        self.root.bind('<Configure>', self.on_window_resize)
        
        # Загрузка избранного
        self.favorites = self.load_favorites()
        
        # Показать топ дня при запуске
        self.show_daily_top()

    def load_data(self):
        """Загрузка данных из CSV файла"""
        try:
            self.df = pd.read_csv('russian_drinks_cleaned.csv')
            self.df.drop_duplicates(subset='Название', inplace=True)
            self.df.fillna('', inplace=True)  
            self.all_ingredients = self._get_all_ingredients()
            
            # Сложность приготовления 
            difficulties = ['Легкий', 'Средний', 'Сложный']
            self.df['Сложность'] = random.choices(difficulties, k=len(self.df))
            
            # Преобразуем DataFrame в словарь для быстрого доступа
            self.cocktails_dict = {row['Название']: row.to_dict() for _, row in self.df.iterrows()}
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось загрузить данные: {str(e)}")
            self.root.destroy()
            return

    def load_favorites(self):
        """Загрузка избранных коктейлей из файла"""
        if os.path.exists('favorites.json'):
            try:
                with open('favorites.json', 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                print(f"Ошибка загрузки избранного: {e}")
                return []
        return []
    
    def save_favorites(self):
        """Сохранение избранных коктейлей в файл"""
        try:
            with open('favorites.json', 'w', encoding='utf-8') as f:
                json.dump(self.favorites, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Ошибка сохранения избранного: {e}")
    
    def _configure_styles(self):
        """Улучшенные стили с увеличенными размерами"""
        # Основные стили
        self.style.configure('Main.TFrame', background=self.bg_color)
        self.style.configure('Secondary.TFrame', background=self.secondary_bg, padding=15)
        
        # Стили для меток
        self.style.configure('TLabel', background=self.secondary_bg, foreground=self.text_color, font=self.normal_font)
        self.style.configure('Title.TLabel', font=self.title_font, foreground=self.accent_color)
        self.style.configure('Subtitle.TLabel', font=self.subtitle_font, foreground=self.text_color)
        
        # Стили для рамок
        self.style.configure(
            'TLabelframe', 
            background=self.secondary_bg, 
            foreground=self.text_color,
            bordercolor=self.accent_color,
            relief='solid',
            borderwidth=3,
            padding=20
        )
        self.style.configure(
            'TLabelframe.Label', 
            background=self.secondary_bg, 
            foreground=self.accent_color,
            font=self.subtitle_font,
            padding=15
        )
        
        # Кнопки
        self.style.configure(
            'TButton',
            background=self.accent_color,
            foreground=self.text_color,
            font=self.button_font,
            borderwidth=2,
            relief='raised',
            padding=(15, 10)
        )
        self.style.map(
            'TButton',
            background=[('active', self.button_hover), ('pressed', self.highlight_color)],
            relief=[('pressed', 'sunken'), ('active', 'raised')]
        )
        
        # Стиль для кнопки избранного
        self.style.configure(
            'Favorite.TButton',
            background=self.favorite_color,
            foreground='#000000',
            font=self.button_font,
            borderwidth=2,
            relief='raised',
            padding=(15, 10)
        )
        
        # Поля ввода
        self.style.configure(
            'TEntry',
            fieldbackground=self.secondary_bg,
            foreground=self.text_color,
            font=self.normal_font,
            padding=12,
            relief='solid',
            bordercolor=self.accent_color
        )
        
        # Таблица
        self.style.configure(
            'Treeview',
            background=self.secondary_bg,
            foreground=self.text_color,
            font=self.normal_font,
            rowheight=35,
            bordercolor=self.accent_color,
            fieldbackground=self.secondary_bg
        )
        self.style.configure(
            'Treeview.Heading',
            background=self.accent_color,
            foreground=self.text_color,
            font=self.button_font,
            padding=10
        )
        self.style.map(
            'Treeview',
            background=[('selected', self.highlight_color)],
            foreground=[('selected', self.text_color)]
        )
        
        # Список ингредиентов
        self.style.configure(
            'Listbox',
            background=self.secondary_bg,
            foreground=self.text_color,
            font=self.normal_font,
            selectbackground=self.highlight_color,
            selectforeground=self.text_color,
            borderwidth=2,
            relief='solid'
        )
        
        # Скроллбары
        self.style.configure(
            'Vertical.TScrollbar',
            background=self.accent_color,
            troughcolor=self.bg_color,
            bordercolor=self.bg_color,
            arrowcolor=self.text_color
        )

    def create_widgets(self):
        """Создание элементов интерфейса"""
        # Основные фреймы
        self.search_frame = ttk.LabelFrame(
            self.scrollable_frame, 
            text=" Поиск коктейлей ",
            style='TLabelframe'
        )
        self.search_frame.grid(row=0, column=0, sticky="ew", padx=10, pady=15)
        
        self.results_frame = ttk.LabelFrame(
            self.scrollable_frame, 
            text=" Результаты поиска ",
            style='TLabelframe'
        )
        self.results_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=15)
        
        self.details_frame = ttk.LabelFrame(
            self.scrollable_frame, 
            text=" Детали коктейля ",
            style='TLabelframe'
        )
        self.details_frame.grid(row=2, column=0, sticky="nsew", padx=10, pady=15)
        
        # Настройка растягивания
        self.scrollable_frame.columnconfigure(0, weight=1)
        self.results_frame.columnconfigure(0, weight=1)
        self.details_frame.columnconfigure(0, weight=1)
        
        # Вкладки поиска
        self.search_notebook = ttk.Notebook(self.search_frame)
        self.search_notebook.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Вкладка поиска по ингредиентам
        self.ingredients_tab = ttk.Frame(self.search_notebook, style='Secondary.TFrame')
        self.search_notebook.add(self.ingredients_tab, text="По ингредиентам")
        
        # Вкладка поиска по названию
        self.name_tab = ttk.Frame(self.search_notebook, style='Secondary.TFrame')
        self.search_notebook.add(self.name_tab, text="По названию")
        
        # Вкладка фильтров
        self.filters_tab = ttk.Frame(self.search_notebook, style='Secondary.TFrame')
        self.search_notebook.add(self.filters_tab, text="Фильтры")
        
        # Элементы поиска по ингредиентам
        ttk.Label(
            self.ingredients_tab, 
            text="Введите ингредиенты через запятую:", 
            style='Subtitle.TLabel'
        ).pack(anchor=tk.W, pady=(0, 15))
        
        self.ingredients_entry = ttk.Entry(
            self.ingredients_tab, 
            style='TEntry'
        )
        self.ingredients_entry.pack(fill=tk.X, pady=15, ipady=8)
        
        # Фрейм для кнопок
        btn_frame = ttk.Frame(self.ingredients_tab, style='Secondary.TFrame')
        btn_frame.pack(fill=tk.X, pady=15)
        
        ttk.Button(
            btn_frame, 
            text="Найти коктейли", 
            command=self.search_cocktails,
            style='TButton'
        ).pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)
        
        ttk.Button(
            btn_frame, 
            text="Топ дня", 
            command=self.show_daily_top,
            style='TButton'
        ).pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)
        
        ttk.Button(
            btn_frame, 
            text="Случайный коктейль", 
            command=self.show_random_cocktail,
            style='TButton'
        ).pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)
        
        # Список ингредиентов
        ttk.Label(
            self.ingredients_tab, 
            text="Доступные ингредиенты:", 
            style='Subtitle.TLabel'
        ).pack(anchor=tk.W, pady=(15, 10))
        
        list_frame = ttk.Frame(self.ingredients_tab, style='Secondary.TFrame')
        list_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        self.ingredients_listbox = tk.Listbox(
            list_frame, 
            height=10,
            selectmode=tk.MULTIPLE,
            font=self.normal_font,
            relief='solid',
            borderwidth=2
        )
        scrollbar = ttk.Scrollbar(list_frame, orient="vertical", command=self.ingredients_listbox.yview)
        self.ingredients_listbox.configure(yscrollcommand=scrollbar.set)
        
        self.ingredients_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        for ing in self.all_ingredients:
            self.ingredients_listbox.insert(tk.END, ing)
            
        self.ingredients_listbox.bind('<<ListboxSelect>>', self.add_selected_ingredient)
        
        # Элементы поиска по названию
        ttk.Label(
            self.name_tab, 
            text="Введите название коктейля:", 
            style='Subtitle.TLabel'
        ).pack(anchor=tk.W, pady=(0, 15))
        
        self.name_entry = ttk.Entry(
            self.name_tab, 
            style='TEntry'
        )
        self.name_entry.pack(fill=tk.X, pady=15, ipady=8)
        
        ttk.Button(
            self.name_tab, 
            text="Найти по названию", 
            command=self.search_by_name,
            style='TButton'
        ).pack(fill=tk.X, pady=15)
        
        # Элементы фильтров
        ttk.Label(
            self.filters_tab, 
            text="Фильтры:", 
            style='Subtitle.TLabel'
        ).pack(anchor=tk.W, pady=(0, 15))
        
        # Фильтр по типу алкоголя
        alcohol_frame = ttk.Frame(self.filters_tab, style='Secondary.TFrame')
        alcohol_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(
            alcohol_frame, 
            text="Тип алкоголя:", 
            style='TLabel'
        ).pack(side=tk.LEFT, padx=10)
        
        self.alcohol_var = tk.StringVar()
        alcohol_types = ['Все'] + sorted(self.df['Тип_алкоголя'].dropna().unique().tolist())
        self.alcohol_combobox = ttk.Combobox(
            alcohol_frame, 
            textvariable=self.alcohol_var, 
            values=alcohol_types,
            state='readonly'
        )
        self.alcohol_combobox.current(0)
        self.alcohol_combobox.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=10)
        
        # Фильтр по сложности
        difficulty_frame = ttk.Frame(self.filters_tab, style='Secondary.TFrame')
        difficulty_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(
            difficulty_frame, 
            text="Сложность:", 
            style='TLabel'
        ).pack(side=tk.LEFT, padx=10)
        
        self.difficulty_var = tk.StringVar()
        difficulties = ['Все', 'Легкий', 'Средний', 'Сложный']
        self.difficulty_combobox = ttk.Combobox(
            difficulty_frame, 
            textvariable=self.difficulty_var, 
            values=difficulties,
            state='readonly'
        )
        self.difficulty_combobox.current(0)
        self.difficulty_combobox.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=10)
        
        ttk.Button(
            self.filters_tab, 
            text="Применить фильтры", 
            command=self.apply_filters,
            style='TButton'
        ).pack(fill=tk.X, pady=15)
        
        # Таблица результатов
        tree_frame = ttk.Frame(self.results_frame, style='Secondary.TFrame')
        tree_frame.pack(fill=tk.BOTH, expand=True)
        
        self.results_tree = ttk.Treeview(
            tree_frame, 
            columns=('name', 'type', 'alcohol', 'difficulty'), 
            show='headings',
            style='Treeview'
        )
        tree_scroll = ttk.Scrollbar(tree_frame, orient="vertical", command=self.results_tree.yview)
        self.results_tree.configure(yscrollcommand=tree_scroll.set)
        
        self.results_tree.heading('name', text='Название')
        self.results_tree.heading('type', text='Тип')
        self.results_tree.heading('alcohol', text='Алкогольный')
        self.results_tree.heading('difficulty', text='Сложность')
        
        # Автоматическое растягивание колонок
        self.results_tree.column('name', width=350, stretch=tk.YES, anchor='w')
        self.results_tree.column('type', width=200, stretch=tk.YES, anchor='w')
        self.results_tree.column('alcohol', width=150, stretch=tk.YES, anchor='w')
        self.results_tree.column('difficulty', width=150, stretch=tk.YES, anchor='w')
        
        self.results_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        self.results_tree.bind('<<TreeviewSelect>>', self.show_cocktail_details)
        
        # Детали коктейля
        details_container = ttk.Frame(self.details_frame, style='Secondary.TFrame')
        details_container.pack(fill=tk.BOTH, expand=True, pady=15)
        
        # Изображение
        img_frame = ttk.Frame(details_container, style='Secondary.TFrame', width=350)
        img_frame.pack(side=tk.LEFT, fill=tk.BOTH, padx=15)
        
        self.cocktail_image = tk.Label(
            img_frame, 
            bg=self.secondary_bg,
            fg=self.text_color,
            font=self.normal_font,
            text="Загрузка изображения...",
            compound=tk.CENTER
        )
        self.cocktail_image.pack(expand=True, fill=tk.BOTH)
        
        # Информация
        info_frame = ttk.Frame(details_container, style='Secondary.TFrame')
        info_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=15)
        
        self.cocktail_name = ttk.Label(
            info_frame, 
            font=self.title_font,
            style='Title.TLabel'
        )
        self.cocktail_name.pack(anchor=tk.W, pady=(0, 20))
        
        self.cocktail_type = ttk.Label(info_frame, style='Subtitle.TLabel')
        self.cocktail_type.pack(anchor=tk.W, pady=(0, 15))
        
        self.cocktail_glass = ttk.Label(info_frame, style='Subtitle.TLabel')
        self.cocktail_glass.pack(anchor=tk.W, pady=(0, 20))
        
        # Кнопки
        btn_frame = ttk.Frame(info_frame, style='Secondary.TFrame')
        btn_frame.pack(fill=tk.X, pady=15)
        
        ttk.Button(
            btn_frame, 
            text="Инструкция", 
            command=self.show_instructions,
            style='TButton'
        ).pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)
        
        ttk.Button(
            btn_frame, 
            text="Ингредиенты", 
            command=self.show_ingredients,
            style='TButton'
        ).pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)
        
        self.favorite_btn = ttk.Button(
            btn_frame, 
            text="В избранное", 
            command=self.toggle_favorite,
            style='Favorite.TButton'
        )
        self.favorite_btn.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)
        
        ttk.Button(
            btn_frame, 
            text="Экспорт", 
            command=self.export_recipe,
            style='TButton'
        ).pack(side=tk.LEFT, expand=True, fill=tk.X, padx=10)
        
        # Текстовое поле
        text_frame = ttk.Frame(info_frame, style='Secondary.TFrame')
        text_frame.pack(fill=tk.BOTH, expand=True)
        
        self.details_text = tk.Text(
            text_frame, 
            wrap=tk.WORD,
            bg=self.secondary_bg,
            fg=self.text_color,
            font=self.normal_font,
            padx=15,
            pady=15,
            insertbackground=self.text_color,
            selectbackground=self.highlight_color,
            borderwidth=2,
            relief='solid'
        )
        text_scroll = ttk.Scrollbar(text_frame, orient="vertical", command=self.details_text.yview)
        self.details_text.configure(yscrollcommand=text_scroll.set)
        
        self.details_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        text_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Прокрутка колесом мыши
        self.canvas.bind_all("<MouseWheel>", self._on_mousewheel)
        
        # Установка минимальных размеров для предотвращения "схлопывания" элементов
        self.search_frame.grid_propagate(False)
        self.results_frame.grid_propagate(False)
        self.details_frame.grid_propagate(False)

    def _on_mousewheel(self, event):
        """Обработка прокрутки колесом мыши"""
        if event.delta:
            self.canvas.yview_scroll(int(-1*(event.delta/120)), "units")
        
    def on_window_resize(self, event):
        """Обработка изменения размера окна"""
        if hasattr(self, 'results_tree'):
            total_width = self.results_tree.winfo_width()
            if total_width > 100:
                self.results_tree.column('name', width=int(total_width*0.4))
                self.results_tree.column('type', width=int(total_width*0.25))
                self.results_tree.column('alcohol', width=int(total_width*0.2))
                self.results_tree.column('difficulty', width=int(total_width*0.15))
    
    def _get_all_ingredients(self):
        """Получение всех уникальных ингредиентов"""
        ingredients = set()
        for i in range(1, 6):
            col = f'Ингредиент{i}'
            for ing in self.df[col].dropna().unique():
                if ing and str(ing).strip() not in ('', '0'):
                    ingredients.add(str(ing).strip())
        return sorted(ingredients)
    
    def _get_daily_top(self, count=5):
        """Получение топовых коктейлей дня"""
        today = datetime.now().date()
        random.seed(today.toordinal())
        return self.df.sample(n=min(count, len(self.df))).to_dict('records')
    
    def _process_image_queue(self):
        """Фоновый поток для обработки изображений"""
        while True:
            url = self.image_queue.get()
            if url is None:  # Сигнал для остановки
                break
            
            if url in self.image_cache:
                # Изображение уже в кэше
                photo = self.image_cache[url]
                if self.current_image_url == url:  # Проверка на актуальность изображения
                    self.root.after(0, self._update_image, photo)
                continue
            
            try:
                response = requests.get(url, timeout=5)
                response.raise_for_status()
                img_data = response.content
                img = Image.open(BytesIO(img_data))
                
                max_size = min(400, self.root.winfo_height() // 3)
                img.thumbnail((max_size, max_size))
                
                photo = ImageTk.PhotoImage(img)
                self.image_cache[url] = photo
                
                if self.current_image_url == url:  # Проверка на актуальность изображения
                    self.root.after(0, self._update_image, photo)
            except Exception as e:
                print(f"Ошибка загрузки изображения: {e}")
                if self.current_image_url == url:
                    self.root.after(0, self._show_image_error)
            
            self.image_queue.task_done()
    
    def _update_image(self, photo):
        """Обновление изображения в интерфейсе"""
        self.cocktail_image.config(image=photo, text='')
        self.cocktail_image.image = photo
    
    def _show_image_error(self):
        """Показать сообщение об ошибке загрузки изображения"""
        self.cocktail_image.config(
            text="Изображение\\nнедоступно", 
            compound=tk.CENTER,
            fg=self.text_color,
            font=self.subtitle_font
        )
        self.cocktail_image.image = None
    
    def add_selected_ingredient(self, event):
        """Добавление выбранного ингредиента в поле ввода"""
        selected = [self.ingredients_listbox.get(i) for i in self.ingredients_listbox.curselection()]
        current_text = self.ingredients_entry.get()
        if current_text:
            new_text = current_text + ", " + ", ".join(selected)
        else:
            new_text = ", ".join(selected)
        self.ingredients_entry.delete(0, tk.END)
        self.ingredients_entry.insert(0, new_text)
        
    def search_cocktails(self):
        """Поиск коктейлей по ингредиентам"""
        ingredients = [ing.strip() for ing in self.ingredients_entry.get().split(',') if ing.strip()]
        
        if not ingredients:
            messagebox.showwarning("Ошибка", "Пожалуйста, введите хотя бы один ингредиент")
            return
            
        mask = pd.Series([True] * len(self.df))
        for ing in ingredients:
            mask &= (self.df['Ингредиент1'].str.contains(ing, case=False, na=False) |
                    self.df['Ингредиент2'].str.contains(ing, case=False, na=False) |
                    self.df['Ингредиент3'].str.contains(ing, case=False, na=False) |
                    self.df['Ингредиент4'].str.contains(ing, case=False, na=False) |
                    self.df['Ингредиент5'].str.contains(ing, case=False, na=False))
        
        results = self.df[mask]
        self.display_results(results)
    
    def search_by_name(self):
        """Поиск коктейлей по названию"""
        name = self.name_entry.get().strip()
        
        if not name:
            messagebox.showwarning("Ошибка", "Пожалуйста, введите название коктейля")
            return
            
        results = self.df[self.df['Название'].str.contains(name, case=False, na=False)]
        self.display_results(results)
    
    def apply_filters(self):
        """Применение фильтров"""
        alcohol_filter = self.alcohol_var.get()
        difficulty_filter = self.difficulty_var.get()
        
        mask = pd.Series([True] * len(self.df))
        
        if alcohol_filter != 'Все':
            mask &= (self.df['Тип_алкоголя'] == alcohol_filter)
            
        if difficulty_filter != 'Все':
            mask &= (self.df['Сложность'] == difficulty_filter)
        
        results = self.df[mask]
        self.display_results(results)
    
    def display_results(self, results):
        """Отображение результатов поиска"""
        for item in self.results_tree.get_children():
            self.results_tree.delete(item)
            
        if results.empty:
            self.results_tree.insert('', tk.END, values=("Ничего не найдено", "", "", ""))
        else:
            for _, row in results.iterrows():
                self.results_tree.insert('', tk.END, values=(
                    row['Название'], 
                    row['Категория'], 
                    row['Тип_алкоголя'],
                    row['Сложность']
                ))
                
        if len(self.results_tree.get_children()) > 0:
            self.results_tree.selection_set(self.results_tree.get_children()[0])
            self.results_tree.focus(self.results_tree.get_children()[0])
            self.show_cocktail_details(None)
    
    def show_daily_top(self):
        """Показать топ коктейлей дня"""
        daily_top = self._get_daily_top()
        results = pd.DataFrame(daily_top)
        self.display_results(results)
    
    def show_random_cocktail(self):
        """Показать случайный коктейль"""
        random_cocktail = self.df.sample(n=1)
        self.display_results(random_cocktail)
    
    def show_cocktail_details(self, event):
        """Показать детали выбранного коктейля"""
        selected_item = self.results_tree.selection()
        if not selected_item:
            return
            
        cocktail_name = self.results_tree.item(selected_item[0], 'values')[0]
        if cocktail_name == "Ничего не найдено":
            return
            
        cocktail = self.cocktails_dict[cocktail_name]
        self.current_cocktail = cocktail
        
        self.cocktail_name.config(text=cocktail['Название'])
        self.cocktail_type.config(text=f"Тип: {cocktail['Категория']} ({cocktail['Тип_алкоголя']})")
        self.cocktail_glass.config(text=f"Бокал: {cocktail['Бокал']}")
        
        # Обновление кнопки избранного
        if cocktail['Название'] in self.favorites:
            self.favorite_btn.config(text="★ В избранном")
        else:
            self.favorite_btn.config(text="☆ В избранное")
        
        # Загрузка изображения
        self.load_cocktail_image(cocktail['Изображение'])
        self.details_text.delete(1.0, tk.END)
    
    def load_cocktail_image(self, url):
        """Загрузка изображения коктейля"""
        # Сначала очищаем текущее изображение и показываем индикатор загрузки
        self.cocktail_image.config(
            image='', 
            text="Загрузка...",
            compound=tk.CENTER,
            fg=self.text_color,
            font=self.subtitle_font
        )
        self.cocktail_image.image = None
        
        if not url or url.strip() == '':
            self._show_image_error()
            return
            
        self.current_image_url = url
        
        # Проверяем кэш
        if url in self.image_cache:
            self._update_image(self.image_cache[url])
        else:
            # Добавляем в очередь на загрузку
            self.image_queue.put(url)
    
    def toggle_favorite(self):
        """Добавление/удаление из избранного"""
        if not hasattr(self, 'current_cocktail'):
            return
            
        cocktail_name = self.current_cocktail['Название']
        
        if cocktail_name in self.favorites:
            self.favorites.remove(cocktail_name)
            self.favorite_btn.config(text="☆ В избранное")
            messagebox.showinfo("Избранное", f"Коктейль {cocktail_name} удалён из избранного")
        else:
            self.favorites.append(cocktail_name)
            self.favorite_btn.config(text="★ В избранном")
            messagebox.showinfo("Избранное", f"Коктейль {cocktail_name} добавлен в избранное")
        
        self.save_favorites()
    
    def show_instructions(self):
        """Показать инструкцию приготовления"""
        if hasattr(self, 'current_cocktail'):
            self.details_text.delete(1.0, tk.END)
            instructions = self.current_cocktail.get('Инструкция', '')
            if not instructions.strip():
                instructions = "Инструкция не указана"
            self.details_text.insert(tk.END, instructions)
    
    def show_ingredients(self):
        if hasattr(self, 'current_cocktail'):
            self.details_text.delete(1.0, tk.END)
            
            ingredients = []
            for i in range(1, 6):
                ing_col = f'Ингредиент{i}'
                measure_col = f'Мера{i}'
                
                ing = self.current_cocktail.get(ing_col, '')
                if pd.isna(ing) or str(ing).strip() in ('', '0'):
                    continue
                
                measure = self.current_cocktail.get(measure_col, '')
                
                if pd.notna(measure) and str(measure).strip() != '':
                    ingredients.append(f"• {str(ing).strip()} - {str(measure).strip()}")
                else:
                    ingredients.append(f"• {str(ing).strip()}")
            
            if ingredients:
                self.details_text.insert(tk.END, "Ингредиенты:\n\n" + "\n".join(ingredients))
            else:
                self.details_text.insert(tk.END, "Ингредиенты не указаны")
    
    def export_recipe(self):
        """Экспорт рецепта в файл"""
        if not hasattr(self, 'current_cocktail'):
            return
            
        # Получаем данные о текущем коктейле
        cocktail = self.current_cocktail
        
        # Собираем текст для экспорта
        export_text = f"Рецепт коктейля: {cocktail['Название']}\\n"
        export_text += f"Тип: {cocktail['Категория']} ({cocktail['Тип_алкоголя']})\\n"
        export_text += f"Бокал: {cocktail['Бокал']}\\n\\n"
        
        # Ингредиенты
        export_text += "Ингредиенты:\\n"
        for i in range(1, 6):
            ing_col = f'Ингредиент{i}'
            measure_col = f'Мера{i}'
            
            ing = cocktail.get(ing_col, '')
            if not ing.strip():
                continue
            
            measure = cocktail.get(measure_col, '')
            
            if measure.strip():
                export_text += f"- {ing.strip()} - {measure.strip()}\\n"
            else:
                export_text += f"- {ing.strip()}\\n"
        
        # Инструкция
        export_text += "\\nИнструкция:\\n"
        instructions = cocktail.get('Инструкция', '')
        if not instructions.strip():
            instructions = "Инструкция не указана"
        export_text += instructions
        
        # Запрашиваем место сохранения файла
        file_path = filedialog.asksaveasfilename(
            defaultextension=".txt",
            filetypes=[("Текстовые файлы", "*.txt"), ("Все файлы", "*.*")],
            title="Сохранить рецепт как"
        )
        
        if file_path:
            try:
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(export_text)
                messagebox.showinfo("Успех", "Рецепт успешно экспортирован")
            except Exception as e:
                messagebox.showerror("Ошибка", f"Не удалось сохранить файл: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    try:
        app = EnhancedCocktailApp(root)
        root.mainloop()
    except Exception as e:
        messagebox.showerror("Ошибка", f"Произошла ошибка: {str(e)}")

Ошибка загрузки изображения: HTTPSConnectionPool(host='www.thecocktaildb.com', port=443): Read timed out.
Ошибка загрузки изображения: HTTPSConnectionPool(host='www.thecocktaildb.com', port=443): Read timed out.
Ошибка загрузки изображения: HTTPSConnectionPool(host='www.thecocktaildb.com', port=443): Read timed out.
Ошибка загрузки изображения: HTTPSConnectionPool(host='www.thecocktaildb.com', port=443): Read timed out.
Ошибка загрузки изображения: HTTPSConnectionPool(host='www.thecocktaildb.com', port=443): Read timed out.
Ошибка загрузки изображения: HTTPSConnectionPool(host='www.thecocktaildb.com', port=443): Read timed out.


Ошибка загрузки изображения: HTTPSConnectionPool(host='www.thecocktaildb.com', port=443): Read timed out.


Exception in thread Thread-18 (_process_image_queue):
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\response.py", line 903, in _error_catcher
    yield
  File "c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\response.py", line 1028, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\response.py", line 1011, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 473, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^